In [1]:
import googlemaps
from geopy import geocoders
from geopy.exc import GeocoderTimedOut
from csv import DictReader
import numpy as np
from scipy import stats
import random

addressData = "data_files/DOLLAR_ADDRESSES.csv"
gmaps = googlemaps.Client(key = "")
geolocator = geocoders.GoogleV3(api_key="")

In [2]:
with open(addressData) as f:
    dg = [row["DG"] for row in DictReader(f)]
with open(addressData) as f:
    dt = [row["DT"] for row in DictReader(f)]    
dt = [i for i in dt if i]

np.save('data_files/dg.npy', dg)
np.save('data_files/dt.npy', dt)
    
dg = random.sample(dg, 100)
dt = random.sample(dt, 100)

In [3]:
dgCoords = list()
dgWalmartCoords = list()
dtCoords = list()
dtWalmartCoords = list()

for address in dg:
    location = geolocator.geocode(address, timeout = 10)
    dgCoords.append((location.latitude, location.longitude))
    walmart_directions = gmaps.places("Walmart", (location.latitude, location.longitude))
    walmart_coords = walmart_directions["results"][0]["geometry"]["location"]
    dgWalmartCoords.append((walmart_coords["lat"], walmart_coords["lng"]))
for address in dt:
    location = geolocator.geocode(address, timeout = 10)
    dtCoords.append((location.latitude, location.longitude))
    walmart_directions = gmaps.places("Walmart", (location.latitude, location.longitude))
    walmart_coords = walmart_directions["results"][0]["geometry"]["location"]
    dtWalmartCoords.append((walmart_coords["lat"], walmart_coords["lng"]))

In [4]:
dg2wmCoords = list()
dt2wmCoords = list()
for x in range(len(dgCoords)):
    dg2wmCoords.append((dgCoords[x], dgWalmartCoords[x]))
for x in range(len(dtCoords)):
    dt2wmCoords.append((dtCoords[x], dtWalmartCoords[x]))

np.save('data_files/dg2wmCoords.npy', dg2wmCoords)
np.save('data_files/dt2wmCoords.npy', dt2wmCoords)

In [5]:
dg2wmDistance = list()
dt2wmDistance = list()
for x in range(len(dg2wmCoords)):
    distance = gmaps.distance_matrix(dg2wmCoords[x][0], dg2wmCoords[x][1])
    dg2wmDistance.append(distance['rows'][0]['elements'][0]['distance']['value']/1000)
for x in range(len(dt2wmCoords)):
    distance = gmaps.distance_matrix(dt2wmCoords[x][0], dt2wmCoords[x][1])
    dt2wmDistance.append(distance['rows'][0]['elements'][0]['distance']['value']/1000)

np.save('data_files/dg2wmDistance.npy', dg2wmDistance)
np.save('data_files/dt2wmDistance.npy', dt2wmDistance)

In [6]:
# https://gist.github.com/vishalkuo/f4aec300cf6252ed28d3
def removeOutliers(x):
    a = np.array(x)
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * 1.5
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    result = a[np.where((a >= quartileSet[0]) & (a <= quartileSet[1]))]
    return result.tolist()

dg2wmDistance = removeOutliers(dg2wmDistance)
dt2wmDistance = removeOutliers(dt2wmDistance)

In [7]:
stats.ttest_ind(dg2wmDistance, dt2wmDistance)

Ttest_indResult(statistic=2.8926943562835423, pvalue=0.004272785409558303)